In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-2. 多重定義

### 5-2-1. 関数の定義（おさらい）

#### コード5-1.	関数定義の例(1)

In [2]:
function add(x, y)
    x + y
end

add (generic function with 1 method)

In [3]:
add(x, y) = x + y

add (generic function with 1 method)

#### コード5-2.	関数定義の例(2)

In [4]:
add(x, y, z) = x + y + z

add (generic function with 2 methods)

In [5]:
function add(x, y, z...)
    add(x + y, z...)
end

add (generic function with 3 methods)

#### コード5-3.	多重定義した関数のメソッドの確認

In [6]:
methods(add)

# 3 methods for generic function "add":
[1] add(x, y) in Main at In[3]:1
[2] add(x, y, z) in Main at In[4]:1
[3] add(x, y, z...) in Main at In[5]:1

### 5-2-2. 型シグニチャ

#### コード5-4.	型シグニチャによる関数の多重定義例 (1)

In [7]:
double(x) = 2x

double (generic function with 1 method)

In [8]:
double(s::AbstractString) = s ^ 2

double (generic function with 2 methods)

#### コード5-5.	`double()` 関数の実行例 (1)

In [9]:
double(100)

200

In [10]:
double(1.23)

2.46

In [11]:
double(π)

6.283185307179586

#### コード5-6.	`double()` 関数の実行例 (2)

In [12]:
double("ABC")

"ABCABC"

In [13]:
double(strip("  ABCD  "))

"ABCDABCD"

#### コード5-7.	型シグニチャによる関数（`double()`）の多重定義例 (2)

In [14]:
double(x, y) = string(double(x), double(y))

double (generic function with 3 methods)

In [15]:
double(x::Number, y::Number) = double(x) + double(y)

double (generic function with 4 methods)

In [16]:
double("ABC", "あいう")

"ABCABCあいうあいう"

In [17]:
double(2, "A")

"4AA"

In [18]:
double("😄", 0.2)

"😄😄0.4"

In [19]:
double(3, π)

12.283185307179586

#### コード5-8.	`Base.typesof()` 関数の例

In [20]:
Base.typesof("ABC", "あいう")

Tuple{String, String}

In [21]:
Base.typesof(2, "A")

Tuple{Int64, String}

In [22]:
Base.typesof("😄", 0.2)

Tuple{String, Float64}

In [23]:
Base.typesof(3, π)

Tuple{Int64, Irrational{:π}}

#### コード5-9.	`Base.typesof()` の結果と型シグニチャのサブタイピング

In [24]:
Tuple{Number, Number} <: Tuple{Any, Any}

true

In [25]:
Tuple{Any, Any} <: Tuple{Number, Number}

false

In [26]:
Base.typesof("ABC", "あいう") <: Tuple{Any, Any}

true

In [27]:
Base.typesof("ABC", "あいう") <: Tuple{Number, Number}

false

In [28]:
Base.typesof(2, "A") <: Tuple{Any, Any}

true

In [29]:
Base.typesof(2, "A") <: Tuple{Number, Number}

false

In [30]:
Base.typesof("😄", 0.2) <: Tuple{Any, Any}

true

In [31]:
Base.typesof("😄", 0.2) <: Tuple{Number, Number}

false

In [32]:
Base.typesof(3, π) <: Tuple{Any, Any}

true

In [33]:
Base.typesof(3, π) <: Tuple{Number, Number}

true

### 5-2-3. 実例

#### コード5-10.	MyTime1.jl

In [34]:
abstract type AbstractTime end

function gethour end
function getminute end
function getsecond end

function Base.show(io::IO, time::AbstractTime)
    print(io,
          string(gethour(time), pad=2),
          ':',
          string(getminute(time), pad=2),
          ':',
          string(getsecond(time), pad=2))
end

struct MyTime <: AbstractTime
    hour::Int
    minute::Int
    second::Int
end

gethour(time::MyTime) = time.hour
getminute(time::MyTime) = time.minute
getsecond(time::MyTime) = time.second

struct MyTime2 <: AbstractTime
    seconds::Int
end

gethour(time::MyTime2) = time.seconds ÷ 3600
getminute(time::MyTime2) = time.seconds ÷ 60 % 60
getsecond(time::MyTime2) = time.seconds % 60

getsecond (generic function with 2 methods)

#### コード5-11.	`MyTime` 型の利用例

In [35]:
mytime = MyTime(12, 34, 56)

12:34:56

In [36]:
string(mytime)

"12:34:56"

#### コード5-12.	`MyTime2` 型の利用例

In [37]:
mytime2 = MyTime2(10000)  # 午前0時の10000秒後は 2時46分40秒

02:46:40

In [38]:
string(mytime2)

"02:46:40"

### 5-2-4. メソッドの曖昧さの解決

#### コード5-13.	型シグニチャによる関数（`double()`）の多重定義例 (3)

In [39]:
double(x::Number, y::Float64) = double(x) + 2double(y)

double (generic function with 5 methods)

In [40]:
double(x::Float64, y::Number) = 2double(x) + double(y)

double (generic function with 6 methods)

#### コード5-14.	`double()` 関数の実行例 (3): メソッドの曖昧さによるエラー例

In [41]:
double(1, 2.0)

10.0

In [42]:
double(1.0, 2)

8.0

In [43]:
double(1.0, 2.0)

LoadError: MethodError: double(::Float64, ::Float64) is ambiguous. Candidates:
  double(x::Number, y::Float64) in Main at In[39]:1
  double(x::Float64, y::Number) in Main at In[40]:1
Possible fix, define
  double(::Float64, ::Float64)

#### コード5-15.	型シグニチャによる関数（`double()`）の多重定義例 (4)

In [44]:
double(x::Float64, y::Float64) = 2double(x) + 2double(y)

double (generic function with 7 methods)

In [45]:
double(1.0, 2.0)

12.0

#### コード5-16.	型シグニチャの不整合による「メソッドが見つからない」エラー例

In [46]:
_double(s::String) = s^2

_double (generic function with 1 method)

In [47]:
_double(strip("   ABCD   "))

LoadError: MethodError: no method matching _double(::SubString{String})
[0mClosest candidates are:
[0m  _double([91m::String[39m) at In[46]:1

### コラム. キーワード引数と多重ディスパッチ

#### コード5-a. キーワード引数と多重ディスパッチ

In [48]:
fn(x, y; z=0.0) = x + y + z

fn (generic function with 1 method)

In [49]:
fn(x, y; u="") = "$(x + y)" * u

fn (generic function with 1 method)

In [50]:
fn(1, 2, u="kg")

"3kg"

In [51]:
fn(1, 2, z=3.0)

LoadError: MethodError: no method matching fn(::Int64, ::Int64; z=3.0)
[0mClosest candidates are:
[0m  fn(::Any, ::Any; u) at In[49]:1[91m got unsupported keyword argument "z"[39m